# 使用 Moorissa 的資料集

Ref: https://github.com/moorissa/medium/blob/master/items-recommender/notebooks/recommendation-MT.ipynb

In [1]:
import time
import numpy as np
import pandas as pd

### 匯入資料

In [2]:
transactions = pd.read_csv('../Playground-dataset/06-Recsys-Dataset/trx_data.csv')

In [3]:
transactions.head()

,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


### 進行資料清理

#### Demo 01

In [4]:
# example 1: split product items
transactions['products'] = transactions['products'].apply(
    lambda x: [int(i) for i in x.split('|')])

In [5]:
transactions.head()

,customerId,products
0,0,[20]
1,1,"[2, 2, 23, 68, 68, 111, 29, 86, 107, 152]"
2,2,"[111, 107, 29, 11, 11, 11, 33, 23]"
3,3,"[164, 227]"
4,5,"[2, 2]"


#### Demo 02

In [6]:
# example 2: transform to series
# 將 products 欄位型態，轉為 pd.Series
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


#### Demo 03

In [7]:
# example 3: organize a given table into a dataframe 
# with customerId, single productId, and purchase count

In [8]:
pd.melt(frame=transactions.head(2).set_index('customerId')['products'].apply(
    pd.Series).reset_index(),
        id_vars=['customerId'],
        value_name='products')[:5]

,customerId,variable,products
0,0,0,20.0
1,1,0,2.0
2,0,1,NaN
3,1,1,2.0
4,0,2,NaN


In [9]:
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(),
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1)[:5]  # 丟掉 temporary variable

,customerId,products
0,0,20.0
1,1,2.0
3,1,2.0
5,1,23.0
7,1,68.0


In [10]:
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(),
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'})[:5]  # 計算購買物品的次數

products
customerId products          
0          20.0             1
1          2.0              2
           23.0             1
           29.0             1
           68.0             2

In [11]:
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(),
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index()[:5]

,customerId,products,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2


In [12]:
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(),
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})[:5]

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2


#### 將資料清理寫成一個函數

In [13]:
# 寫成一個函數，來進行資料轉換
def trx_transform(df):
    s = time.time()
    data = pd.melt(frame=df.set_index("customerId")["products"].apply(
        pd.Series).reset_index(),
                  id_vars=["customerId"],
                  value_name="products") \
    .dropna().drop(["variable"], axis=1) \
    .groupby(["customerId", "products"]) \
    .agg({"products": "count"}) \
    .rename(columns={"products":"purchase_count"}) \
    .reset_index()\
    .rename(columns={"products":"productId"})
    data["productId"] = data["productId"].astype(np.int64)
    
    print("Execution Time:", round((time.time() - s)/60, 2), "minutes")
    return data

In [14]:
data = trx_transform(transactions)

Execution Time: 0.21 minutes


In [15]:
print(data.shape)
data.head()

(133585, 3)


,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


### 將每一個 user 的 purchase count 標準化

#### Demo 04

In [10]:
# example 4: transform的使用，使用下面的公式，保留dataframe的原有長度，才可以append回原本的df
# Ref: https://www.jianshu.com/p/509d7b97088c
data.groupby("customerId")["purchase_count"].transform(lambda x: x.max())[:5]

0    5
1    5
2    5
3    5
4    5
Name: purchase_count, dtype: int64

In [11]:
data["max_count"] = data.groupby("customerId")["purchase_count"].transform(
    lambda x: x.max())

In [12]:
data.shape

(133585, 4)

In [13]:
data.head()

,customerId,productId,purchase_count,max_count
0,0,1,2,5
1,0,13,1,5
2,0,19,3,5
3,0,20,1,5
4,0,31,2,5


In [14]:
data.tail()

,customerId,productId,purchase_count,max_count
133580,28596,211,3,3
133581,28596,255,1,3
133582,28598,212,1,1
133583,28604,282,1,1
133584,28605,92,1,1


In [15]:
data["purchase_count_norm"] = data["purchase_count"] / data["max_count"]

In [16]:
data.head()

,customerId,productId,purchase_count,max_count,purchase_count_norm
0,0,1,2,5,0.4
1,0,13,1,5,0.2
2,0,19,3,5,0.6
3,0,20,1,5,0.2
4,0,31,2,5,0.4


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133585 entries, 0 to 133584
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   customerId           133585 non-null  int64  
 1   productId            133585 non-null  int64  
 2   purchase_count       133585 non-null  int64  
 3   max_count            133585 non-null  int64  
 4   purchase_count_norm  133585 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 5.1 MB


In [19]:
data.to_csv("../Playground-dataset/06-Recsys-Dataset/trx_data_norm.csv", index=False)